In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

import warnings
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

In [2]:
%cd ../

/home/chervovn04/Programming/hackathons/2022/digital_breakout_885303


In [3]:
df = pd.read_csv("data/train.csv")

/tmp/ipykernel_12797/1366839329.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/train.csv")


In [4]:
df = df.sample(1_000_000)

In [ ]:
df.head(3) 

In [ ]:
df.info(0) 

In [ ]:
df.drop(columns=['mailrank'], inplace=True) 

In [ ]:
df.index_oper = df.index_oper.replace(' ', '0').astype(float)

In [ ]:
df.index_oper.hist(bins=30)

In [ ]:
sns.heatmap(np.abs(df.corr()), square = True)

In [ ]:
df.shape, df[np.abs(df.weight - df.weight_mfi) <= 10000].shape

In [ ]:
(df[np.abs(df.weight - df.weight_mfi) <= 10000].weight - df.weight_mfi).hist(bins=100)

In [ ]:
df.info(0)

In [ ]:
plt.scatter(df[(df.price_mfi != 0) & (df.transport_pay != 0)].price_mfi, df[(df.price_mfi != 0) & (df.transport_pay != 0)].transport_pay) 

In [ ]:
plt.hist(df.total_qty_over_index, bins=30);

In [ ]:
plt.hist(df.dist_qty_oper_login_1, bins=30);

In [ ]:
df.index_oper = df.index_oper.replace(' ', '0').astype(float).astype(int) 

In [ ]:
Counter(df.index_oper).most_common(10)

In [ ]:
index_data = df[df.index_oper == 102976]

In [ ]:
(df == 0).sum()

In [ ]:
(df.total_qty_over_index == 0).sum()

In [ ]:
Counter(index_data.total_qty_over_index)

In [5]:
original_data = df.copy() 

In [6]:
df.head() 

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
4733911,2313290,1_-1,102971.0,Цех,7506.0,N,0.0,N,N,254.0,...,181.0,76138189.0,8747.0,76146936.0,84110459.0,0,1,0,0,0
2564772,1990648,1016_-1,420306,Цех,7503.0,N,0.0,Y,N,479.0,...,361.0,9570832.0,3258747.0,12829579.0,18959055.0,0,0,0,0,0
5071969,3353185,0_-1,420312.0,Цех,7503.0,N,0.0,Y,N,180.0,...,105.0,4273111.0,1459081.0,5732192.0,5735070.0,0,0,0,0,0
5114803,2342043,1_-1,625962.0,Участок,7503.0,N,0.0,Y,N,125.0,...,43.0,2345548.0,3004.0,2348552.0,19642841.0,0,0,0,0,0
2565378,1255816,1_-1,102971,Цех,7506.0,N,0.0,N,N,223.0,...,184.0,67541214.0,7870.0,67549084.0,72981134.0,0,0,1,0,0


In [8]:
df = df[['oper_type + oper_attr', 'label']]

In [11]:
df['oper_type'], df['oper_attr'] = zip(*df['oper_type + oper_attr'].map(lambda x: x.split('_')))

In [13]:
df.drop(columns=['oper_type + oper_attr'], inplace=True) 

In [22]:
for oattr in [x[0] for x in Counter(df.oper_attr).most_common(df.oper_attr.nunique())]:
    print(oattr, (df.oper_attr == oattr).sum(), df.label[df.oper_attr == oattr].mean())

-1 815700 0.02262964325119529
0 66577 0.0
1 30218 0.26070553974452315
6 26459 0.06553535658944026
2 23883 4.187078675208307e-05
3 19764 0.0
13 11667 0.0
10 1648 0.0
5 1402 0.0
1001 765 0.0
4 564 0.0
20 371 0.0
9 280 0.0
1000 226 0.0
18 167 0.0
21 127 0.0
8 78 0.01282051282051282
182 75 0.7733333333333333
16 24 0.0
22 2 0.0
7 1 0.0
19 1 0.0
11 1 0.0


In [7]:
for otype in [x[0] for x in Counter(df.oper_type).most_common(df.oper_type.nunique())][:20]:
    print(otype, (df.oper_type == otype).sum(), df.label[df.oper_type == otype].mean())

AttributeError: 'DataFrame' object has no attribute 'oper_type'